Date: 10/07/2023

Creation of combined RPE-PR and single cell models using different CS-models (Lukowski & Voight datasets) and the current Human1 model

The models are saved in the folder: combined_models\combined_models_June2023


In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [17]:
# load RPE, PR, Recon3D, and basis Human1 models from cs mods folder
from pathlib import Path
from cobra.io import read_sbml_model, load_matlab_model
mod = read_sbml_model(Path().cwd() / 'cs_mods' / 'Human-GEM_28062023.xml' )
mod_RPE = read_sbml_model(Path().cwd() /  'cs_mods' / 'model_1_RPE__VoigtEtAl2019_.xml')
mod_PRrod = read_sbml_model(Path().cwd() /  'cs_mods' / 'model_2_RodPhotoreceptors__LiangEtAl_2019_.xml' )
mod_PRcone = read_sbml_model(Path().cwd() /  'cs_mods' / 'model_3_ConePhotoreceptors__LiangEtAl_2019_.xml' )
mod_PRrod_Lukowski = read_sbml_model(Path().cwd() /  'cs_mods' / 'model_1_AverageRodPRC0,C1,C2,C3,C4,C7(%).xml' )
mod_PRrod_Lukowski.jname = 'AverageRodPR_Lukowski' 
mod_PRrod_Lukowski.id = 'AverageRodPR_Lukowski' 
mod_PRcone_Lukowski = read_sbml_model(Path().cwd() /  'cs_mods' / 'model_6_C10ConePR(%).xml' )
mod_PRcone_Lukowski.name = 'C10ConePR_Lukowski' 
mod_PRcone_Lukowski.id = 'C10ConePR_Lukowski' 

'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.


In [4]:
# modify context specific models
from src.modify_model import add_all_EX_rxns, remove_compartment

for m in [mod_RPE, mod_PRcone, mod_PRrod, mod_PRrod_Lukowski, mod_PRcone_Lukowski]: # add ATP DM hydrolysis and lost EX rxns to context specific models
    m.add_reactions([mod.reactions.get_by_id('MAR03964').copy()]) # make sure it contains ATP hydrolysis rxn
    m = add_all_EX_rxns(mod,m) # add lost EX rxns
    for r in [r for r in m.reactions if len(r.products)==0]: 
        r.bounds = (0,1000) # close lb for exchange reactions

In [5]:
# remove nucleus ('n') from photoreceptor models     
mod_PRcone = remove_compartment(mod_PRcone.copy(), 'n')  # remove nucleus ('n') from PR model
mod_PRrod = remove_compartment(mod_PRrod.copy(), 'n')  # remove nucleus ('n') from PR model
mod_PRrod_Lukowski = remove_compartment(mod_PRrod_Lukowski.copy(), 'n')  # remove nucleus ('n') from PR model
mod_PRcone_Lukowski = remove_compartment(mod_PRcone_Lukowski.copy(), 'n')  # remove nucleus ('n') from PR model

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpt4ld0foi.lp
Reading time = 0.02 seconds
: 5149 rows, 13072 columns, 49224 nonzeros


c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp05l85989.lp
Reading time = 0.02 seconds
: 5075 rows, 12756 columns, 47716 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpqia4ea6q.lp
Reading time = 0.02 seconds
: 5267 rows, 13206 columns, 49254 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpmv2o4yvd.lp
Reading time = 0.02 seconds
: 5287 rows, 12948 columns, 47916 nonzeros


In [6]:
    from src.modify_model import  add_id_suffix

    # add suffix to all ids in RPE and PR models
    mod_PRrod = add_id_suffix(mod_PRrod, '_PR')
    mod_PRcone = add_id_suffix(mod_PRcone, '_PR')
    mod_PRcone_Lukowski = add_id_suffix(mod_PRcone_Lukowski, '_PR')
    mod_PRrod_Lukowski = add_id_suffix(mod_PRrod_Lukowski, '_PR')
    mod_RPE = add_id_suffix(mod_RPE, '_RPE')

In [7]:
    # add RPE-PR interface reactions and metabolites to RPE and PR models
    from src.combine_RPE_PR import add_interface_RPE, add_interface_PR
    mod_RPE = add_interface_RPE(mod_RPE, 'e_RPE')
    mod_PRrod = add_interface_PR(mod_PRrod, 'e_PR')
    mod_PRcone = add_interface_PR(mod_PRcone, 'e_PR')
    mod_PRrod_Lukowski = add_interface_PR(mod_PRrod_Lukowski, 'e_PR')
    mod_PRcone_Lukowski = add_interface_PR(mod_PRcone_Lukowski, 'e_PR')

In [8]:
    # fuse RPE and PR models   
    from src.modify_model import fix_compartment_dict, add_rxns2groups
    from src.combine_RPE_PR import fuse_RPE_PR
    mod_RPE_PRrod = fuse_RPE_PR(mod_RPE.copy(),mod_PRrod)
    mod_RPE_PRcone = fuse_RPE_PR(mod_RPE.copy(),mod_PRcone)
    mod_RPE_PRrod_Lukowski = fuse_RPE_PR(mod_RPE.copy(),mod_PRrod_Lukowski)
    mod_RPE_PRcone_Lukowski = fuse_RPE_PR(mod_RPE.copy(),mod_PRcone_Lukowski)

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp6ska6ptl.lp
Reading time = 0.03 seconds
: 7186 rows, 19642 columns, 65480 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpf44icidw.lp
Reading time = 0.03 seconds
: 7186 rows, 19642 columns, 65480 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp0o7o7hsr.lp
Reading time = 0.03 seconds
: 4980 rows, 12484 columns, 46510 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp9cb92nuc.lp
Reading time = 0.03 seconds
: 7186 rows, 19642 columns, 65480 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp3tkwcf9p.lp
Reading time = 0.04 seconds
: 7186 rows, 19642 columns, 65480 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpcc4s7pnk.lp
Reading time = 0.02 seconds
: 5055 rows, 12802 columns, 48012 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp23fd9biq.lp
Reading time =

In [9]:
from src.combine_RPE_PR import del_RPE_PR_dupl_rxns

mod_RPE_PRrod = fix_compartment_dict(mod_RPE_PRrod)
mod_RPE_PRrod = add_rxns2groups(mod_RPE_PRrod)
mod_RPE_PRrod = del_RPE_PR_dupl_rxns(mod_RPE_PRrod)

mod_RPE_PRcone = fix_compartment_dict(mod_RPE_PRcone)
mod_RPE_PRcone = add_rxns2groups(mod_RPE_PRcone)    
mod_RPE_PRcone = del_RPE_PR_dupl_rxns(mod_RPE_PRcone)

mod_RPE_PRrod_Lukowski = fix_compartment_dict(mod_RPE_PRrod_Lukowski)
mod_RPE_PRrod_Lukowski = add_rxns2groups(mod_RPE_PRrod_Lukowski)
mod_RPE_PRrod_Lukowski = del_RPE_PR_dupl_rxns(mod_RPE_PRrod_Lukowski)

mod_RPE_PRcone_Lukowski = fix_compartment_dict(mod_RPE_PRcone_Lukowski)
mod_RPE_PRcone_Lukowski = add_rxns2groups(mod_RPE_PRcone_Lukowski)
mod_RPE_PRcone_Lukowski = del_RPE_PR_dupl_rxns(mod_RPE_PRcone_Lukowski)

c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")
c:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [10]:
# remove PR exchange reactions from RPE-PR models
for m in [mod_RPE_PRrod, mod_RPE_PRcone, mod_RPE_PRrod_Lukowski, mod_RPE_PRcone_Lukowski]:
    m.remove_reactions([r.id for r in  m.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) 

In [11]:
# save RPE-PR models
from cobra.io import write_sbml_model
write_sbml_model(mod_RPE_PRrod, 'cs_mods/July2023/mod_RPE_PRrod_Liang.xml')
write_sbml_model(mod_RPE_PRcone, 'cs_mods/July2023/mod_RPE_PRcone_Liang.xml')
write_sbml_model(mod_RPE_PRrod_Lukowski, 'cs_mods/July2023/mod_RPE_PRrod_Lukowski.xml')
write_sbml_model(mod_RPE_PRcone_Lukowski, 'cs_mods/July2023/mod_RPE_PRcone_Lukowski.xml')

In [12]:
def create_single_model(model, to_remove):
    """
    Create single model from RPE-PR model

    Parameters
    ----------
    m : cobra.Model
        RPE-PR model
    to_remove : str
        cell to remove from RPE-PR model
    
    Returns
    -------
    cobra.Model
        RPE model
    
    """
    m = model.copy()
    reactions_to_remove = []
    compartments = [c for c in list(mod_RPE_PRcone_Lukowski.compartments) if to_remove in c]
    for r in m.reactions:
        # list of compartments in reaction that are not in the RPE compartment list
        l = [x for x in list(r.compartments) if x not in compartments]
        # remove reaction if it contains a compartment that is not in the RPE compartment list
        if len(l) > 0:
            reactions_to_remove.append(r.id)

    m.remove_reactions(reactions_to_remove)
    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    return m   

In [13]:
# create single cell models from RPE-PR models
mod_RPE_PRrod_singleRPE = create_single_model(mod_RPE_PRrod, 'PR')
mod_RPE_PRcone_singleRPE = create_single_model(mod_RPE_PRcone, 'PR')
mod_RPE_PRrod_singlePR = create_single_model(mod_RPE_PRrod, 'RPE')
mod_RPE_PRcone_singlePR = create_single_model(mod_RPE_PRcone, 'RPE')

mod_RPE_PRrod_Lukowski_singleRPE = create_single_model(mod_RPE_PRrod_Lukowski, 'PR')
mod_RPE_PRcone_Lukowski_singleRPE = create_single_model(mod_RPE_PRcone_Lukowski, 'PR')
mod_RPE_PRrod_Lukowski_singlePR = create_single_model(mod_RPE_PRrod_Lukowski, 'RPE')
mod_RPE_PRcone_Lukowski_singlePR = create_single_model(mod_RPE_PRcone_Lukowski, 'RPE')


Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpru6xl580.lp
Reading time = 0.05 seconds
: 9560 rows, 25444 columns, 105230 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmppctqyijn.lp
Reading time = 0.06 seconds
: 9628 rows, 25764 columns, 106740 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpeypvwh4s.lp
Reading time = 0.05 seconds
: 9560 rows, 25444 columns, 105230 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp1wwyb5gh.lp
Reading time = 0.05 seconds
: 9628 rows, 25764 columns, 106740 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpg40j1oze.lp
Reading time = 0.05 seconds
: 9738 rows, 25856 columns, 106648 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpkl7l3nqf.lp
Reading time = 0.06 seconds
: 9774 rows, 25606 columns, 105314 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpyrr2bw_6.lp
Reading 

In [14]:
# save RPE-PR models
from cobra.io import write_sbml_model
write_sbml_model(mod_RPE_PRrod_singleRPE, 'cs_mods/July2023/mod_RPE_PRrod_Liang_singleRPE.xml')
write_sbml_model(mod_RPE_PRcone_singleRPE, 'cs_mods/July2023/mod_RPE_PRcone_Liang_singleRPE.xml')
write_sbml_model(mod_RPE_PRrod_singlePR, 'cs_mods/July2023/mod_RPE_PRrod_Liang_singlePR.xml')
write_sbml_model(mod_RPE_PRcone_singlePR, 'cs_mods/July2023/mod_RPE_PRcone_Liang_singlePR.xml')

write_sbml_model(mod_RPE_PRrod_Lukowski_singleRPE, 'cs_mods/July2023/mod_RPE_PRrod_Lukowski_singleRPE.xml')
write_sbml_model(mod_RPE_PRcone_Lukowski_singleRPE, 'cs_mods/July2023/mod_RPE_PRcone_Lukowski_singleRPE.xml')
write_sbml_model(mod_RPE_PRrod_Lukowski_singlePR, 'cs_mods/July2023/mod_RPE_PRrod_Lukowski_singlePR.xml')
write_sbml_model(mod_RPE_PRcone_Lukowski_singlePR, 'cs_mods/July2023/mod_RPE_PRcone_Lukowski_singlePR.xml')


In [15]:
# function to create one long list from a list of lists
def flatten_list(l):
    return [item for sublist in l for item in sublist]

# to check whether the right compartments have been removed from the single cell models
set(flatten_list([list(r.compartments) for r in mod_RPE_PRrod_singleRPE.reactions]))

{'c_PR', 'e_RPE_PR', 'g_PR', 'i_PR', 'l_PR', 'm_PR', 'r_PR', 'x_PR'}